In [1]:
from Bio.Blast import NCBIWWW
from Bio import SeqIO
import json
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import xmltodict
from tqdm import tqdm
import requests

In [6]:
nicotine = '../data/uniprot_results/uniprot-compressed_true_download_true_format_json_query__28Nicotine_-2023.05.11-18.33.15.22.json'

In [7]:

with open(nicotine, 'r') as file:
    data = json.load(file)

# Now you can work with the JSON data stored in the 'data' variable.


In [8]:
ex = data['results'][0]

In [9]:
len(data['results'])

214

In [10]:
uniq = []
for i in data['results']:
  typ = (i['organism']['lineage'][11])

  if typ not in uniq:
    uniq.append(typ)
print(uniq)

['lamiids']


In [11]:
for i in ex['comments']:
    if i['commentType'] == 'SUBCELLULAR LOCATION':
        cell_location = (i['subcellularLocations'][0]['location']['value'])

In [12]:
for i in range(len(data['results'])):
    ex = data['results'][i]
# entry = {}

# #protein names
# entry['uniprot_id'] = 
# entry['primary_accession'] = ex['primaryAccession']
# entry['common_name'] = ex['organism']['commonName']
# entry['species_name'] =
# entry['sequence'] =
# #subcellular location
# for i in ex['comments']:
#     if i['commentType'] == 'SUBCELLULAR LOCATION':
#         entry['cell_location'] = (i['subcellularLocations'][0]['location']['value'])

# #taxonomy
# entry['superkingdom'] = 
# entry['kingdom'] =
# entry['order'] = 
# entry['family'] = ex['organism']['lineage'][13]
# entry['genus'] = 

# #protein properties
# entry['properties'] = ex['features']
    url = "http://localhost:8004/protein/"  # Replace with the appropriate API URL


    payload = {
        "primary_accession":ex['uniProtkbId'],
        "sequence":  ex['sequence']['value'],
        "scientific_name":  ex['organism']['scientificName'],
        "superkingdom": ex['organism']['lineage'][0],
        "kingdom":  ex['organism']['lineage'][1],
        "order": ex['organism']['lineage'][12],
        "genus": ex['organism']['lineage'][14]
    }

    # Send the POST request to the API
    response = requests.post(url, json=payload)

    # Check the response status and print the result
    if response.status_code == 200:
        print("Data uploaded successfully")
    else:
        print(f"Error uploading data: {response.text}")

Data uploaded successfully
Data uploaded successfully
Data uploaded successfully
Data uploaded successfully
Data uploaded successfully
Data uploaded successfully
Data uploaded successfully
Data uploaded successfully
Data uploaded successfully
Data uploaded successfully
Data uploaded successfully
Data uploaded successfully
Data uploaded successfully
Data uploaded successfully
Data uploaded successfully
Data uploaded successfully
Data uploaded successfully
Data uploaded successfully
Data uploaded successfully
Data uploaded successfully
Data uploaded successfully
Data uploaded successfully
Data uploaded successfully
Data uploaded successfully
Data uploaded successfully
Error uploading data: {"detail":[{"loc":[],"msg":"duplicate key value violates unique constraint \"protein_sequence_key\"\nDETAIL:  Key (sequence)=(MVVSEKSKILIIGGTGYIGKYLVETSAKSGHPTFALIRESTLKNPEKSKLIDTFKSYGVTLLFGDISNQESLLKAIKQVDVVISTVGGQQFTDQVNIIKAIKEAGNIKRFLPSEFGFDVDHARAIEPAASLFALKVRIRRMIEAEGIPYTYVICNWFADFFLPNLGQLEAKTPPRDK

In [13]:
for i in tqdm(ex['features']):
    print('type: ', i['type'])
    print('desc : ', i['description'])
    if 'ligand' in i:
        print('liga : ', i['ligand']['name'])

100%|██████████| 13/13 [00:00<00:00, 63255.16it/s]

type:  Chain
desc :  Diamine oxidase [copper-containing] 1, peroxisomal
type:  Active site
desc :  Proton acceptor
type:  Active site
desc :  Schiff-base intermediate with substrate; via topaquinone
type:  Binding site
desc :  
liga :  substrate
type:  Binding site
desc :  
liga :  substrate
type:  Binding site
desc :  
liga :  Cu cation
type:  Binding site
desc :  
liga :  Cu cation
type:  Binding site
desc :  
liga :  Mn(2+)
type:  Binding site
desc :  
liga :  Mn(2+)
type:  Binding site
desc :  
liga :  Cu cation
type:  Site
desc :  Microbody targeting signal
type:  Modified residue
desc :  2',4',5'-topaquinone
type:  Disulfide bond
desc :  


In [14]:
from concurrent.futures import ProcessPoolExecutor
import os
from Bio import SeqIO
from Bio.Blast import NCBIWWW
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from tqdm import tqdm

def process_protein(ex):
    already_blasted = os.listdir('./blast_results/')
    if f"{ex['primaryAccession']}_blast_results.xml" not in already_blasted:
        protein_sequence_str = ex['sequence']['value']
        protein_sequence = SeqRecord(Seq(protein_sequence_str), id="my_protein", description="My protein sequence")
        result_handle = NCBIWWW.qblast("blastp", "nr", protein_sequence.seq)
        with open(f"./blast_results/{ex['primaryAccession']}_blast_results.xml", "w") as result_file:
            print(f'writing results for {ex}')
            result_file.write(result_handle.read())


In [ ]:
with ProcessPoolExecutor(max_workers=8) as executor:
    executor.map(process_protein, (data['results']))